#### Realizar una desanonimización de los datos de Medallion y números de licencia. Subir a Moodle el código utilizado (incluyendo explicaciones y comentarios) y el dataset desanonimizado (2 ficheros en total).

In [1]:
import pandas as pd
import numpy as np
import hashlib 
import itertools

Empezamos haciendo un diccionario para los posibles valores de medallions y licencias y sus correspondientes hashes.

#### Medallion

In [5]:
#medallion
#inicializamos tabla
hashes = {}

#añadimos datos de los medallion con formato 5X55
alfabeto = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' #valores que puede tomar X

for num1 in range(10):
    for num2 in range(100):
        for x in alfabeto:
            #crea el hash y lo almacena
            codigomed = str(num1)+x+('%0.2d' % num2)
            hashes[hashlib.md5(codigomed.encode()).hexdigest().upper()] = codigomed

In [6]:
#medallions con formato XX555
for num1 in range(1000):
    for comb in itertools.permutations(alfabeto, 2):
        #crea el hash y lo añade
        codigomed = ''.join(comb)+('%0.3d' % num1)
        hashes[hashlib.md5(codigomed.encode()).hexdigest().upper()] = codigomed 

In [7]:
#medallions con formato XXX555
for num1 in range(1000):
    for comb in itertools.permutations(alfabeto, 3):
        #crea el hash y lo añade
        codigomed = ''.join(comb)+('%0.3d' % num1)
        hashes[hashlib.md5(codigomed.encode()).hexdigest().upper()]=codigomed

#### Licencias

In [8]:
#lo mismo para licencias
for num in range(1000000):
    #crea el hash y lo añade
    codigolic = '%0.6d' % num
    codigolic2 = str(5)+codigolic
    hashes[hashlib.md5(codigolic.encode()).hexdigest().upper()]=codigolic
    hashes[hashlib.md5(codigolic2.encode()).hexdigest().upper()]=codigolic2     

A continuación, recorremos el dataset original y buscamos en el diccionario de hashes cada medallion y hack_license para intentar identificarlos.

In [9]:
df= pd.read_csv("trip_data.csv")
df.head(3)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,789E7E64EC7C31A88C1683D3BA163A46,3C0FB2975708DF2E873C6D7C72E7EABE,VTS,1,NaN,2013-12-05 22:45:00,2013-12-05 22:54:00,4,540,2.08,-73.973854,40.762615,-73.954041,40.775455
1,9C34910D26A3CBBC48D309CEA303A1B7,D05DEF3DD93BBCF5D596476C8C18A138,VTS,1,NaN,2013-12-05 22:53:00,2013-12-05 22:57:00,6,240,0.56,-73.982300,40.766106,-73.982079,40.763973
2,5BA6F6CB6D998B34BAD3701A118E5419,AA6850948D1690435B1339BA4DCC0AA8,VTS,1,NaN,2013-12-05 22:51:00,2013-12-05 22:55:00,1,240,1.44,-73.989204,40.757675,-73.974815,40.765179


In [10]:
df_id = df.copy()

In [11]:
#desanonimizamos y almacenamos las nuevas columnas
medallion2 = [None]*len(df)
license2 = [None]*len(df)

for i in df_id.index:
    hashmed = df_id.loc[i]['medallion']
    valmed = hashes.get(hashmed)
    medallion2[i] = valmed

    hashlic = df_id.loc[i]['hack_license']
    vallic = hashes.get(hashlic)
    license2[i] = vallic

In [14]:
#añadimos las nuevas columnas al dataset de copia
df_id['medallion_id']=medallion2
df_id['license_id']=license2

In [15]:
#cambiamos de orden las columnas para ver mejor los valores desanonimizados
df_id.columns

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'medallion_id', 'license_id'],
      dtype='object')

In [16]:
df_id = df_id.reindex(columns=['medallion', 'hack_license', 'medallion_id', 'license_id','vendor_id',
                         'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime','passenger_count',
                         'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude',
                         'dropoff_longitude', 'dropoff_latitude'])
df_id.head(3)

,medallion,hack_license,medallion_id,license_id,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,789E7E64EC7C31A88C1683D3BA163A46,3C0FB2975708DF2E873C6D7C72E7EABE,6D62,5123537,VTS,1,NaN,2013-12-05 22:45:00,2013-12-05 22:54:00,4,540,2.08,-73.973854,40.762615,-73.954041,40.775455
1,9C34910D26A3CBBC48D309CEA303A1B7,D05DEF3DD93BBCF5D596476C8C18A138,4D22,5171753,VTS,1,NaN,2013-12-05 22:53:00,2013-12-05 22:57:00,6,240,0.56,-73.982300,40.766106,-73.982079,40.763973
2,5BA6F6CB6D998B34BAD3701A118E5419,AA6850948D1690435B1339BA4DCC0AA8,7G79,491738,VTS,1,NaN,2013-12-05 22:51:00,2013-12-05 22:55:00,1,240,1.44,-73.989204,40.757675,-73.974815,40.765179


Vemos el número de hashes no reidentificados.

In [29]:
print('No se han identificado '+str(len(np.unique(df_id['medallion'][df_id['medallion_id'].isnull()])))+' medallions, que intervinieron en '+str(np.sum(df_id['medallion_id'].isnull()))+' viajes.')

No se han identificado 14 medallions, que intervinieron en 101 viajes.


In [32]:
print('No se han identificado '+str(len(np.unique(df_id['hack_license'][df_id['license_id'].isnull()])))+' licencias, que intervinieron en '+str(np.sum(df_id['license_id'].isnull()))+' viajes.')

No se han identificado 11 licencias, que intervinieron en 45 viajes.


In [31]:
#guardamos el nuevo dataset en un archivo csv
df_id.to_csv('df_desanon.csv')